In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import time

In [2]:
import pandas as pd

firstData = True
def parseRates(browser):
    global firstData
    data = pd.read_html(browser.find_element_by_class_name("rb").get_attribute('outerHTML'))
    data = data[0]
    data = data[2:19] # drop columns 0, 1 and row 20
    del data[0]
    del data[2]
    del data[3]
    data.columns = ["name", "date", "USD_Առք", "USD_Վաճ", "EUR_Առք", "EUR_Վաճ", "RUB_Առք", "RUB_Վաճ", "GBF_Առք", "GBF_Վաճ"]
    if firstData:
        with open('rates.csv', 'w', encoding="utf-8") as f:
            data.to_csv(f, header=True, index=False)
            firstData = False
    else:
        with open('rates.csv', 'a', encoding="utf-8") as f:
            data.to_csv(f, header=False, index=False)

In [3]:
browser = webdriver.Firefox()

In [4]:
url = 'http://rate.am/en/armenian-dram-exchange-rates/banks/non-cash'
browser.get(url)

In [5]:
datePicker = browser.find_element_by_link_text("Rates by previous date")
datePicker.click()

In [6]:
years = browser.find_element_by_name("YearSelector")
yearsCnt = len(years.find_elements_by_tag_name("option"))
mntCnt = 12
daysCnt = 31
hours = browser.find_element_by_id("ctl00_Content_RB_dtpick1_dpHours")
hoursCnt = len(hours.find_elements_by_tag_name("option"))

In [7]:
for year in range(yearsCnt):
    for mnt in range(mntCnt):
        for day in range(1, daysCnt):
            for hour in range(hoursCnt):
                datePicker = browser.find_element_by_link_text("Rates by previous date")
                datePicker.click()
                time.sleep(.5)
                
                years = browser.find_element_by_name("YearSelector")
                selectYear = Select(years)
                selectYear.select_by_index(year)
                time.sleep(.5)

                mnts = browser.find_element_by_name("MonthSelector")
                selectMnt = Select(mnts)
                selectMnt.select_by_index(mnt)
                time.sleep(.5)
                try:
                    dayLbl = browser.find_element_by_xpath("//td[@class='calTD' and text()='" + str(day) + "']")
                    dayLbl.click() 
                    time.sleep(.5)                   
                except:
                    try:
                        dayLbl = browser.find_element_by_xpath("//font[@class='calR' and text()='" + str(day) + "']")
                        dayLbl.click() 
                        time.sleep(.5)  
                    except:
                        #print("trying year:" + str(year)+" mnt:"+ str(mnt) + " day:" + str(day) + " hour:" +str(hour) + " FAILED")
                        break

                hours = browser.find_element_by_id("ctl00_Content_RB_dtpick1_dpHours")
                selectHour = Select(hours)
                selectHour.select_by_index(hour)
                time.sleep(.5)

                submitBtn = browser.find_element_by_id("ctl00_Content_RB_dtpick1_btnOk")
                submitBtn.click()

                parseRates(browser)
                # be nice...
                time.sleep(15)
                #print("trying year:" + str(year)+" mnt:"+ str(mnt) + " day:" + str(day) + " hour:" +str(hour) + " SUCSESS")

KeyboardInterrupt: 

In [8]:
browser.close()